In [21]:
import pandas as pd
import numpy as np
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Reshape, Embedding,Bidirectional, LSTM, Dropout, Flatten,concatenate, Dense, BatchNormalization, Lambda, TimeDistributed, Dot, dot
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.model_selection import train_test_split

from zipfile import ZipFile
from os.path import expanduser, exists

import datetime
import time
import json

In [22]:
train_dataset = pd.read_csv("/home/sagar24/nlp_dataset/quora/archive/questions.csv")
train_dataset

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [23]:
train_df_copy = train_dataset.copy()
train_df_copy.describe()

,id,qid1,qid2,is_duplicate
count,404351.000000,404351.000000,404351.000000,404351.000000
mean,202175.000000,391840.987691,390195.973765,0.369248
std,116726.223686,228430.857607,228803.645742,0.482602
min,0.000000,1.000000,2.000000,0.000000
25%,101087.500000,193381.000000,191012.000000,0.000000
50%,202175.000000,390630.000000,388364.000000,0.000000
75%,303262.500000,589514.000000,588071.000000,1.000000
max,404350.000000,789800.000000,789801.000000,1.000000


In [24]:
train_df_copy[train_df_copy['is_duplicate'] > 0].describe()

,id,qid1,qid2,is_duplicate
count,149306.000000,149306.000000,149306.000000,149306.0
mean,200837.169457,391048.269266,390776.485078,1.0
std,116197.758898,227527.280271,227507.463550,0.0
min,5.000000,11.000000,12.000000,1.0
25%,100183.000000,193421.000000,193116.000000,1.0
50%,200479.500000,390273.000000,389638.500000,1.0
75%,301292.750000,587967.000000,587675.000000,1.0
max,404347.000000,789794.000000,789795.000000,1.0


In [25]:
len(train_df_copy[train_df_copy['is_duplicate'] > 0]['qid1'].unique())

148487

In [26]:
len(train_df_copy['qid1'].unique())

399334

In [27]:
train_df = train_dataset.copy()

In [28]:
q1_set = set(train_df['question1'].unique())
q2_set = set(train_df['question2'].unique())
all_ques_list = q1_set | q2_set
len(all_ques_list)

537388

In [29]:
q1_list = train_df['question1'].tolist()
q1_list = [str(ques) for ques in q1_list]
q2_list = train_df['question2'].tolist()
q2_list = [str(ques) for ques in q2_list]
is_duplicate_list = train_df['is_duplicate'].tolist()

print(q1_list[0],":",q2_list[0],":",is_duplicate_list[0])

What is the step by step guide to invest in share market in india? : What is the step by step guide to invest in share market? : 0


In [30]:
all_questions_list = q1_list + q2_list
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(all_questions_list)

q1_word_seq = tokenizer.texts_to_sequences(q1_list)
q2_word_seq = tokenizer.texts_to_sequences(q2_list)
word_index = tokenizer.word_index

print("Words in index: %d" % len(word_index))

Words in index: 95603


In [31]:
# Save the tokenizer word index we've gotten for later

dictionary = word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [32]:


embeddings_index = {}

with open(expanduser('/home/sagar24/nlp codes/8542_11957_compressed_glove.6B.50d.txt/glove.6B.50d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 400000


In [33]:
len(embeddings_index['the'])

50

In [34]:
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 50


nb_words = min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 35261


In [35]:
MAX_SEQUENCE_LENGTH = 70
#to reduce training time
q1_data = pad_sequences(q1_word_seq, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(q2_word_seq, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(is_duplicate_list, dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data tensor: (404351, 70)
Shape of question2 data tensor: (404351, 70)
Shape of label tensor: (404351,)


In [36]:
X = np.stack((q1_data, q2_data), axis=1)
y = labels
X.shape

(404351, 2, 70)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]
Q1_train.shape

(323480, 70)

In [38]:
from tensorflow.keras.layers import add
question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(nb_words + 1, 
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix], 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(question1)
q1 = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True), merge_mode="sum")(q1)

q2 = Embedding(nb_words + 1, 
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix], 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(question2)
q2 = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True), merge_mode="sum")(q2)

attention = dot([q1,q2], [1,1])
attention = Flatten()(attention)
attention = Dense((MAX_SEQUENCE_LENGTH*EMBEDDING_DIM))(attention)
attention = Reshape((MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))(attention)
DROPOUT=0.2
merged = add([q1,attention])
merged = Flatten()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[question1,question2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 70)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 70)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 70, 50)       4780200     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 70, 50)       4780200     input_4[0][0]                    
_______________________________________________________________________________________

In [40]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint('question_pairs_weights_type1_final_new.h5', monitor='val_acc', save_best_only=True)]
history = model.fit([Q1_train, Q2_train],
                    y_train,
                    epochs=20,
                    validation_data=([Q1_test, Q2_test], y_test),
                    verbose=1,
                    batch_size=512,
                    callbacks=callbacks)
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Starting training at 2021-07-02 13:08:32.100073
Epoch 1/20
632/632 [==============================] - 365s 578ms/step - loss: 0.5806 - accuracy: 0.6878 - val_loss: 0.5156 - val_accuracy: 0.7395
Epoch 2/20
632/632 [==============================] - 362s 572ms/step - loss: 0.4877 - accuracy: 0.7511 - val_loss: 0.4954 - val_accuracy: 0.7445
Epoch 3/20
632/632 [==============================] - 360s 570ms/step - loss: 0.4552 - accuracy: 0.7715 - val_loss: 0.4520 - val_accuracy: 0.7764
Epoch 4/20
632/632 [==============================] - 360s 570ms/step - loss: 0.4315 - accuracy: 0.7867 - val_loss: 0.4460 - val_accuracy: 0.7792
Epoch 5/20
632/632 [==============================] - 360s 570ms/step - loss: 0.4128 - accuracy: 0.7983 - val_loss: 0.4281 - val_accuracy: 0.7906
Epoch 6/20
632/632 [==============================] - 361s 572ms/step - loss: 0.3942 - accuracy: 0.8100 - val_loss: 0.4260 - val_accuracy: 0.7902
Epoch 7/20
632/632 [==============================] - 361s 572ms/step - loss

In [ ]:
model.save_weights('quo_qn_pair_bilstm.h5')

In [41]:
#for testing :
from keras.preprocessing.text import text_to_word_sequence

def convert_text_to_index_array(text, dictionary):
    words = text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

Using TensorFlow backend.


In [42]:

question1 = "When was the company formed?"
question2 = "When is the company anniversary?"

q1_word_seq = convert_text_to_index_array(question1,dictionary)
q1_word_seq = [q1_word_seq]
q2_word_seq = convert_text_to_index_array(question2,dictionary)
q2_word_seq = [q2_word_seq]
q1_data = pad_sequences(q1_word_seq, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(q2_word_seq, maxlen=MAX_SEQUENCE_LENGTH)

pred = model.predict([q1_data,q2_data])
print(pred)

[[0.878805]]


In [43]:

question1 = "What is a dog?"
question2 = "What is a cat?"

q1_word_seq = convert_text_to_index_array(question1,dictionary)
q1_word_seq = [q1_word_seq]
q2_word_seq = convert_text_to_index_array(question2,dictionary)
q2_word_seq = [q2_word_seq]
q1_data = pad_sequences(q1_word_seq, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(q2_word_seq, maxlen=MAX_SEQUENCE_LENGTH)

pred = model.predict([q1_data,q2_data])
print(pred)

[[0.26883698]]
